<a href="https://colab.research.google.com/github/thevedantt/VibeMatcher-Nexora-AI/blob/main/VibeMatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
"""
🧠 VIBE MATCHER PROTOTYPE — AI at Nexora
======================================================================

📍 Introduction:
At Nexora, AI embodies the combination of engineering, creativity, and empathy—values that I really believe in.
I've learned how intelligent systems can go beyond automation to produce meaningful, human-centered experiences
through my work on projects like Exagram and VaidAI. I find inspiration in Nexora's use of technology to understand
people as well as solve issues by transforming data into context-aware, adaptable solutions. My objective as an
AI engineer is to support this vision by developing models that respond naturally, think intelligently, and make
every encounter seem smooth and meaningful.

======================================================================
"""

import numpy as np
import pandas as pd
import json
import time

# ================================
# Load Product Catalog
# ================================
with open("/content/fashion-product-data_json.json", "r") as f:
    data = json.load(f)

catalog = pd.DataFrame(data)

# ---------- Ensure Required Columns ----------
if "name" not in catalog.columns:
    catalog["name"] = [f"Product_{i}" for i in range(len(catalog))]

if "description" not in catalog.columns:
    catalog["description"] = catalog["name"]

if "vibe_tags" not in catalog.columns:
    catalog["vibe_tags"] = catalog["description"].apply(
        lambda d: [w.lower() for w in str(d).split()[:3]]
    )

if "price" not in catalog.columns:
    catalog["price"] = np.random.randint(50, 250, size=len(catalog))

if "category" not in catalog.columns:
    catalog["category"] = "misc"

# ================================
# Embedding Simulation
# ================================
def simulate_embedding(text, dim=1536):
    np.random.seed(abs(hash(text)) % (10 ** 6))
    return np.random.rand(dim)

# ================================
# Query Expansion
# ================================
class QueryExpander:
    def __init__(self):
        self.expansion_map = {
            "urban": ["city", "street", "modern"],
            "cozy": ["warm", "comfortable", "snug"],
            "bold": ["strong", "confident", "statement"],
            "rebellious": ["edgy", "nonconformist", "streetwear"],
            "energetic": ["active", "dynamic", "sporty"],
            "autumn": ["fall", "seasonal", "warm"],
            "boho": ["bohemian", "earthy", "festival"],
        }

    def expand(self, query):
        expanded = query.lower()
        for key, words in self.expansion_map.items():
            if key in expanded:
                expanded += " " + " ".join(words)
        return expanded if expanded != query.lower() else query, expanded != query.lower()

# ================================
# Matcher
# ================================
class VibeMatcher:
    def __init__(self, catalog_df):
        self.catalog = catalog_df.copy()
        self.embeddings = []
        self.expander = QueryExpander()
        self.embedding_dim = 1536
        self.init_embeddings()

    def init_embeddings(self):
        for _, row in self.catalog.iterrows():
            desc = str(row.get("description", ""))
            emb = simulate_embedding(desc, self.embedding_dim)
            self.embeddings.append(emb)
        self.embeddings = np.vstack(self.embeddings)

    def cosine_similarity(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def match(self, query, top_k=3, query_tags=None, query_price=None):
        start = time.time()
        expanded_query, expanded = self.expander.expand(query)
        query_emb = simulate_embedding(expanded_query, self.embedding_dim)

        sims = [self.cosine_similarity(query_emb, emb) for emb in self.embeddings]
        sims = np.array(sims)

        tag_boost = np.zeros(len(self.catalog))
        if query_tags:
            for i, tags in enumerate(self.catalog["vibe_tags"]):
                overlap = len(set(tags) & set(query_tags))
                tag_boost[i] = overlap * 0.05
        sims += tag_boost

        if query_price:
            price_diff = np.abs(self.catalog["price"] - query_price)
            price_penalty = np.exp(-price_diff / 100)
            sims *= price_penalty

        indices = np.argsort(-sims)[:top_k]
        scores = sims[indices]
        results = self.catalog.iloc[indices].copy()
        results["score"] = scores
        results["quality"] = results["score"].apply(
            lambda s: "Excellent" if s > 0.85 else ("Good" if s > 0.7 else "Fair")
        )

        latency = time.time() - start
        return results, scores, {
            "expanded_query": expanded_query,
            "expansion_applied": expanded,
            "latency": latency,
            "weak_match": all(s < 0.7 for s in scores),
        }

matcher = VibeMatcher(catalog)

# ================================
# Interactive Testing
# ================================
print("\nInitializing Vibe Matcher System...")
print("=" * 70)
print(f"✅ Loaded {len(catalog)} products from catalog.")
print(f"📂 Categories: {catalog['category'].unique().tolist()}")
print("\n⚙️ Generating product embeddings...")
print(f"✅ Generated {len(matcher.embeddings)} embeddings")
print(f"Embedding dimension: {matcher.embedding_dim}")
print("=" * 70)
print("✅ Engine ready with advanced modules:")
print("  • Multi-Modal Fusion Scoring (semantic + tags + price)")
print("  • Contextual Query Expansion")
print("=" * 70)

# Take 1 query at a time
print("\n💬 Enter a vibe query:")
query = input("Query: ").strip()
if not query:
    query = "energetic urban chic"

print("\n🧾 Testing the following query:")
print(f" 1. {query}")
print("\n" + "-" * 70)

start_time = time.time()
auto_tags = [v for v in matcher.expander.expansion_map.keys() if v in query.lower()]
if auto_tags:
    print(f"Inferred vibe tags: {auto_tags}")
else:
    print("No explicit tags detected.")
results, scores, meta = matcher.match(query, top_k=3, query_tags=auto_tags or None)
latency = meta["latency"]
if meta["expansion_applied"]:
    print(f"Expanded → {meta['expanded_query']}")
if meta["weak_match"]:
    print(" Weak match detected (score < 0.7)")
print(f"Latency: {latency * 1000:.2f} ms\n")

for rank, (_, row) in enumerate(results.iterrows(), 1):
    print(f"{rank}. {row['name']} — {row['quality']} ({row['score']:.3f}) | ${row['price']}")
print("-" * 70)

print("\n" + "=" * 70)
print("📊 PERFORMANCE SUMMARY")
print("=" * 70)
print(f"Latency: {latency * 1000:.2f} ms")
print(f"Success Rate (>0.7): {0 if meta['weak_match'] else 1}/1")
print("=" * 70)



Initializing Vibe Matcher System...
✅ Loaded 25 products from catalog.
📂 Categories: ['misc']

⚙️ Generating product embeddings...
✅ Generated 25 embeddings
Embedding dimension: 1536
✅ Engine ready with advanced modules:
  • Multi-Modal Fusion Scoring (semantic + tags + price)
  • Contextual Query Expansion

💬 Enter a vibe query:
Query:  energetic urban chic

🧾 Testing the following query:
 1. energetic urban chic

----------------------------------------------------------------------
Inferred vibe tags: ['urban', 'energetic']
Expanded → energetic urban chic city street modern active dynamic sporty
Latency: 1.78 ms

1. Product_2 — Good (0.759) | $136
2. Product_16 — Good (0.758) | $60
3. Product_17 — Good (0.757) | $176
----------------------------------------------------------------------

📊 PERFORMANCE SUMMARY
Latency: 1.78 ms
Success Rate (>0.7): 1/1
